In [1]:
# Importing modules
import pandas as pd
import os
# Read data into papers
papers = pd.read_csv('R Process Data.csv')
# Print head
papers.head()

,text,id
0,Just got home from staying here.,202083785
1,Just opened this month. Barely a dozen guests ...,202411626
2,I was in Atlanta for a conference for several ...,203700076
3,A couple of 30-somethings on a girls weekend a...,204048976
4,Stayed here last week for 1 night. Great locat...,204990867


In [28]:
papers.rename(columns={'text': 'paper_text'}, inplace=True)
papers['paper_text'] = papers['paper_text'].astype(str)
# Load the regular expression library
import re
# Remove punctuation
papers['paper_text_processed'] = \
papers['paper_text'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
papers['paper_text_processed'] = \
papers['paper_text_processed'].map(lambda x: x.lower())
# Print out the first rows of papers
papers['paper_text_processed'].head()

0                      just got home from staying here
1    just opened this month barely a dozen guests h...
2    i was in atlanta for a conference for several ...
3    a couple of 30-somethings on a girls weekend a...
4    stayed here last week for 1 night great locati...
Name: paper_text_processed, dtype: object

In [34]:
#papers['paper_text_processed'] = papers.paper_text_processed.str.replace(r'\b(\w{1,2})\b', '') 
#papers['paper_text_processed'] = papers['paper_text_processed'].replace(r'\W*\b\w{1,2}\b')

papers['paper_text_processed'] = papers.paper_text_processed.str.replace(r'\b(\w{1,2})\b', '')
#shortword = re.compile(r'\W*\b\w{1,2}\b')
#print(shortword.sub('', papers['paper_text_processed']))

papers

/home/jhala.d/.conda/envs/jhala.d/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


,paper_text,id,paper_text_processed
0,Just got home from staying here.,202083785,just got home from staying here
1,Just opened this month. Barely a dozen guests ...,202411626,just opened this month barely dozen guests he...
2,I was in Atlanta for a conference for several ...,203700076,was atlanta for conference for several days...
3,A couple of 30-somethings on a girls weekend a...,204048976,couple -somethings girls weekend and this ...
4,Stayed here last week for 1 night. Great locat...,204990867,stayed here last week for night great locatio...
...,...,...,...
2031596,1,"2,336",
2031597,1,"2,336",
2031598,1,"2,336",
2031599,1,"2,336",


In [36]:
papers['paper_text_processed']

0                            just got home from staying here
1          just opened this month barely  dozen guests he...
2           was  atlanta for  conference for several days...
3           couple  -somethings   girls weekend and this ...
4          stayed here last week for  night great locatio...
                                 ...                        
2031596                                                     
2031597                                                     
2031598                                                     
2031599                                                     
2031600                                                     
Name: paper_text_processed, Length: 2031601, dtype: object

In [37]:
import gensim
from gensim.utils import simple_preprocess
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data = papers.paper_text_processed.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

['just', 'got', 'home', 'from', 'staying', 'here']


In [41]:
data_words[:1][0][:30]

['just', 'got', 'home', 'from', 'staying', 'here']

In [42]:
# NLTK Stop words
# import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


In [43]:
import spacy
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)


In [47]:
with open('papers.pkl', 'wb') as handle:
    pickle.dump(papers, handle)

In [46]:
import pickle
with open('words_without_stopwords.pkl', 'wb') as handle:
    pickle.dump(data_words_nostops, handle)

In [51]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
#def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
#    texts_out = []
#    for sent in texts:
#        doc = nlp(" ".join(sent)) 
#        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
#    return texts_out

#data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
#print(data_lemmatized[:1])

In [53]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words_nostops)
# Create Corpus
texts = data_words_nostops
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])


[[(0, 1), (1, 1), (2, 1)]]


In [54]:
with open('corpus.pkl', 'wb') as handle:
    pickle.dump(corpus, handle)

In [2]:
import pickle
with open('corpus.pkl', 'rb') as handle:
    corpus = pickle.load(handle)

In [55]:
# Build LDA model
import gensim
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       num_topics=13, 
                                       random_state=100,
                                       chunksize=1000)

from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
#doc_lda = lda_model[corpus]

/home/jhala.d/.conda/envs/jhala.d/lib/python3.7/site-packages/gensim/models/ldamodel.py:846: RuntimeWarning: divide by zero encountered in double_scalars
  perwordbound = self.bound(chunk, subsample_ratio=subsample_ratio) / (subsample_ratio * corpus_words)


[(0,
  '0.033*"188" + 0.016*"25" + 0.009*"12" + 0.008*"75" + 0.008*"24" + '
  '0.008*"422" + 0.007*"81" + 0.007*"129" + 0.006*"233" + 0.006*"162"'),
 (1,
  '0.023*"188" + 0.018*"75" + 0.017*"25" + 0.010*"129" + 0.010*"63" + '
  '0.009*"55" + 0.008*"422" + 0.008*"38" + 0.007*"12" + 0.007*"78"'),
 (2,
  '0.020*"188" + 0.018*"25" + 0.012*"75" + 0.011*"63" + 0.010*"15" + '
  '0.010*"129" + 0.008*"81" + 0.006*"38" + 0.006*"123" + 0.006*"55"'),
 (3,
  '0.018*"25" + 0.011*"8221" + 0.009*"188" + 0.009*"2226" + 0.008*"6533" + '
  '0.008*"15" + 0.007*"422" + 0.007*"8053" + 0.007*"4415" + 0.006*"129"'),
 (4,
  '0.027*"25" + 0.017*"188" + 0.011*"63" + 0.010*"55" + 0.009*"15" + '
  '0.008*"12" + 0.007*"129" + 0.007*"47" + 0.006*"81" + 0.006*"24"'),
 (5,
  '0.028*"25" + 0.015*"129" + 0.012*"63" + 0.011*"75" + 0.010*"81" + '
  '0.008*"110" + 0.007*"77" + 0.007*"233" + 0.007*"188" + 0.006*"55"'),
 (6,
  '0.034*"25" + 0.023*"188" + 0.015*"129" + 0.014*"75" + 0.012*"55" + '
  '0.010*"77" + 0.009*"24" + 

In [ ]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

In [6]:
#with open('lda_model_k5.pkl', 'wb') as handle:
#    pickle.dump(lda_model, handle)

In [57]:
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
LDAvis_prepared

/home/jhala.d/.conda/envs/jhala.d/lib/python3.7/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.040677 -0.013628       1        1  24.064716
5     -0.023799 -0.015461       2        1  15.220391
8     -0.013106 -0.010022       3        1  13.435651
1     -0.039539 -0.017342       4        1   9.402638
11     0.206578 -0.002015       5        1   9.195523
6     -0.031125 -0.003297       6        1   5.767846
12    -0.022381 -0.009441       7        1   5.710762
10    -0.024405 -0.002250       8        1   5.264120
9     -0.004351 -0.010830       9        1   3.774914
4     -0.011989  0.006054      10        1   2.300259
3     -0.006645  0.131418      11        1   2.277962
7      0.018716 -0.045406      12        1   2.040462
2     -0.007277 -0.007779      13        1   1.544757, topic_info=          Term          Freq         Total Category  logprob  loglift
25       hotel  2.065521e+06  2.065521e+06  Default  30.0000  30.0000
188       room  1.585951e+06  1.585951e+06  Default  29.0000  29.0000
75       great  8.128100e+05  8.128100e+05  Default  28.0000  28.0000
63       staff  7.499290e+05  7.499290e+05  Default  27.0000  27.0000
15     atlanta  4.125010e+05  4.125010e+05  Default  26.0000  26.0000
..         ...           ...           ...      ...      ...      ...
43        desk  4.623879e+03  2.855052e+05  Topic13  -5.6472   0.0473
233      would  5.086442e+03  5.294680e+05  Topic13  -5.5519  -0.4750
90   breakfast  4.544335e+03  3.360459e+05  Topic13  -5.6646  -0.1331
24        good  4.457969e+03  4.687835e+05  Topic13  -5.6838  -0.4851
447       well  4.305163e+03  3.021174e+05  Topic13  -5.7186  -0.0807

[1265 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
9142       1  0.075272  aber
9142       2  0.037833  aber
9142       3  0.026798  aber
9142       4  0.004204  aber
9142       5  0.090116  aber
...      ...       ...   ...
6970      11  0.027446  этом
6970      12  0.024217  этом
6970      13  0.008072  этом
75707     10  0.934345   しかし
75690      9  0.963929   冷蔵庫

[6025 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 6, 9, 2, 12, 7, 13, 11, 10, 5, 4, 8, 3])

/home/jhala.d/.conda/envs/jhala.d/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/jhala.d/.conda/envs/jhala.d/lib/python3.7/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/home/jhala.d/.conda/envs/jhala.d/lib/python3.7/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/home/jhala.d/.conda/envs/jhala.d/lib/python3.7/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/home/jhala.d/.conda/envs/jhala.d/lib/python3.7/site-packages/setuptools/_distutils/version.p

In [8]:
#calculating model perplexity
perplexity = lda_model.log_perplexity(corpus)

#printing model perplexity
print(perplexity)

-7.626286056846199


In [25]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words_nostops, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

NameError: name 'id2word' is not defined

In [ ]:
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared


In [ ]:
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
LDAvis_prepared

In [35]:
LDAvis_prepared

NameError: name 'LDAvis_prepared' is not defined